# Subset Generation

In [12]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd

## Interpolated Subset (Subset-296)

songdo_traffic_core.dataset.nodelink 모듈을 통해 생성한 데이터를 기반으로 Interpolation 및 subset 생성

In [ ]:
from songdo_traffic_core.dataset.interpolator import (
    IterativeRandomForestInterpolator,
    LinearInterpolator,
    SplineInterpolator,
)
from songdo_traffic_core.dataset.metr_imc.generator import MetrImcSubsetGenerator

In [ ]:
def extend_nans_around_zeros(series: pd.Series) -> pd.Series:
    series = series.copy()
    nan_indices = series[series.isna()].index

    for idx in nan_indices:
        idx_pos = series.index.get_loc(idx)

        i = idx_pos - 1
        while i >= 0 and series.iat[i] == 0:
            series.iat[i] = np.nan
            i -= 1

        i = idx_pos + 1
        while i < len(series) and series.iat[i] == 0:
            series.iat[i] = np.nan
            i += 1

    return series

In [ ]:
df_imc: pd.DataFrame = pd.read_hdf("../datasets/metr-imc/metr-imc.h5")
df_imc

현재는 결측치가 너무 많은 데이터는 제외하고 실험

In [ ]:
no_missing_columns = df_imc.columns[df_imc.isnull().sum() == 0].to_list()
less_500_missing_columns = df_imc.columns[df_imc.isnull().sum() < 500].to_list()    # 현재 선택된 대상 그룹
less_750_missing_columns = df_imc.columns[df_imc.isnull().sum() < 750].to_list()

In [ ]:
generator = MetrImcSubsetGenerator(
    nodelink_dir="../datasets/metr-imc/nodelink",
    imcrts_dir="../datasets/metr-imc/imcrts",
    metr_imc_dir="../datasets/metr-imc/",
)

결측치 주변의 값이 대부분 0으로 되어 있다. 결측치가 발생하고 측정이 재개되었을 때, 시간이 걸리고 0으로 측정된다는 가설하에 결측치 주변의 0은 모두 결측치로 처리하였다.

In [ ]:
df = generator.metr_imc_df
df = df.sort_index()
generator.metr_imc_df = df.apply(extend_nans_around_zeros)

Interpolation 방법은 여러가지가 있을 수 있으며 현재는 아래와 같은 Interpolation으로 정의

In [ ]:
interpolator = SplineInterpolator()

### Interpolation 완료한 Subset 생성

In [9]:
SUBSET_TARGET_DIR = "../datasets/metr-imc/subsets/metr-imc-296-int"

In [ ]:
generator.generate_subset(less_500_missing_columns, SUBSET_TARGET_DIR, interpolator)

In [5]:
pd.read_hdf("../datasets/metr-imc/subsets/metr-imc-296-int/metr-imc.h5").to_excel(
    "../datasets/metr-imc/subsets/metr-imc-296-int/metr-imc-296-int.xlsx"
)

생성된 데이터를 Shapefile로 시각화

In [10]:
from songdo_traffic_core.dataset.metr_imc.converter.graph_sensor_locations import SensorNetworkView

SUBSET_TARGET_DISTANCE_FILE_PATH = os.path.join(SUBSET_TARGET_DIR, "distances_imc_2023.csv")
SUBSET_TARGET_SENSOR_FILE_PATH = os.path.join(SUBSET_TARGET_DIR, "graph_sensor_locations.csv")
SUBSET_TARGET_NETWORK_DIR = os.path.join(SUBSET_TARGET_DIR, "miscellaneous")

SensorNetworkView(SUBSET_TARGET_DISTANCE_FILE_PATH, SUBSET_TARGET_SENSOR_FILE_PATH).to_file(SUBSET_TARGET_NETWORK_DIR)



100%|██████████| 48195/48195 [00:00<00:00, 56929.86it/s]
2024/07/22 11:11:59 pyogrio._io [INFO] Created 296 records
2024/07/22 11:11:59 pyogrio._io [INFO] Created 47,899 records


## Small Group (Subset-37)

앞에서 선택한 결측치 500개 미만의 데이터 중 임의로 Clustering을 한 37개의 그룹을 추가로 추출하여 작은 그룹을 만들었다. 이 그룹이 적절한 그래프 구조와 적은 결측치를 가지고 있기에 테스트 용도로 적합할 것으로 판단하였다.

아래는 QGIS를 통해서 추출한 Subset 목록의 Shapefile 이다.

In [15]:
subset_37_targets = gpd.read_file("../datasets/metr-imc/subsets/metr-imc-37/selected_roads.shp")
subset_37_targets.head()

,index,NODE_ID,geometry
0,71,1650026001,POINT (126.7067 37.39807)
1,1191,1640014100,POINT (126.68411 37.40726)
2,1363,1640014900,POINT (126.66937 37.41002)
3,2615,1640049000,POINT (126.69901 37.42607)
4,3035,1640012000,POINT (126.67751 37.40461)
